# Fashion MNIST 분류를 위한 MLP 모델 구현 및 시각화 (PyTorch)이 노트북은 Fashion MNIST 데이터셋을 사용하여 다층 퍼셉트론(MLP) 모델을 구축, 학습, 평가하고 그 과정을 다양하게 시각화합니다.

In [ ]:
# ## 1. 라이브러리 임포트import torchimport torch.nn as nnimport torch.optim as optimimport torch.nn.functional as Ffrom torch.utils.data import DataLoaderimport torchvisionimport torchvision.transforms as transformsimport matplotlib.pyplot as pltimport numpy as npfrom sklearn.metrics import confusion_matriximport seaborn as snsimport time  # 학습 시간 측정을 위해 추가

## 2. 하이퍼파라미터 및 설정

In [ ]:
# 하이퍼파라미터 설정batch_size = 128         # 배치 크기를 조금 늘려 학습 속도 개선 시도learning_rate = 0.001num_epochs = 15         # 에포크 수를 조금 늘려 충분한 학습 유도hidden_size1 = 512hidden_size2 = 256# Device 설정 (GPU 우선 사용)device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')print(f"Using device: {device}")# 재현성을 위한 랜덤 시드 설정SEED = 42np.random.seed(SEED)torch.manual_seed(SEED)if torch.cuda.is_available():    torch.cuda.manual_seed_all(SEED)

## 3. Fashion MNIST 데이터셋 로드 및 전처리- `torchvision.datasets.FashionMNIST`를 사용하여 데이터를 로드합니다.- `transforms.ToTensor()`: 이미지를 PyTorch Tensor로 변환합니다.- `transforms.Normalize((0.5,), (0.5,))`: 이미지를 -1 ~ 1 범위로 정규화합니다. (평균 0.5, 표준편차 0.5 기준)

In [ ]:
# 데이터셋 로드 및 전처리를 위한 Transform 정의# 데이터 증강(Augmentation)을 추가하여 일반화 성능 향상 시도 (예: RandomHorizontalFlip)transform_train = transforms.Compose([    transforms.RandomHorizontalFlip(p=0.5), # 50% 확률로 좌우 반전    transforms.ToTensor(),    transforms.Normalize((0.2860,), (0.3530,)) # FashionMNIST의 실제 평균/표준편차 값 사용 (근사치)])transform_test = transforms.Compose([    transforms.ToTensor(),    transforms.Normalize((0.2860,), (0.3530,)) # FashionMNIST의 실제 평균/표준편차 값 사용 (근사치)])# 데이터셋 다운로드 및 생성train_dataset = torchvision.datasets.FashionMNIST(    root='./data',    train=True,    download=True,    transform=transform_train # 훈련 데이터에는 증강 적용)test_dataset = torchvision.datasets.FashionMNIST(    root='./data',    train=False,    download=True,    transform=transform_test # 테스트 데이터에는 증강 미적용)

## 4. DataLoader 생성- 학습 및 테스트 데이터셋을 배치 단위로 처리하기 위한 DataLoader를 생성합니다.- `shuffle=True` (훈련 데이터): 에포크마다 데이터 순서를 섞어 모델이 데이터 순서에 의존하지 않도록 합니다.

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2) # num_workers 추가로 데이터 로딩 속도 개선 시도test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)print(f"Number of training batches: {len(train_loader)}")print(f"Number of test batches: {len(test_loader)}")

## 5. 클래스 이름 정의 및 데이터 시각화- Fashion MNIST 데이터셋의 10개 클래스 이름을 정의합니다.- 첫 번째 배치의 이미지 샘플과 해당 레이블을 시각화하여 데이터 형태를 확인합니다.

In [ ]:
# Fashion MNIST 클래스 이름classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
# 데이터 시각화 함수 (정규화 복원 기능 추가)def imshow_with_denormalize(img, ax, title):    # 정규화 복원: img * std + mean    # 주의: 사용된 Normalize 값에 맞춰야 함 (여기서는 (0.2860,), (0.3530,))    mean = 0.2860    std = 0.3530    img = img * std + mean    npimg = img.numpy()    ax.imshow(np.transpose(npimg, (1, 2, 0)).squeeze(), cmap='gray') # (C, H, W) -> (H, W, C)    ax.set_title(title)    ax.axis('off')def show_images_grid(images, labels, classes, num_examples=20):    num_rows = (num_examples + 4) // 5  # 5열 기준으로 행 수 계산    fig, axes = plt.subplots(num_rows, 5, figsize=(12, 3 * num_rows))    axes = axes.flatten()    for i in range(min(num_examples, len(images))):        imshow_with_denormalize(images[i], axes[i], classes[labels[i]])    # 남는 subplot 비활성화    for j in range(i + 1, len(axes)):        axes[j].axis('off')    plt.tight_layout()    plt.show()

In [ ]:
# 훈련 데이터 샘플 시각화 실행print("Displaying training data samples...")examples = iter(train_loader)example_data, example_targets = next(examples)show_images_grid(example_data[:20], example_targets[:20], classes, num_examples=20)

## 6. MLP 모델 정의- `nn.Module`을 상속받아 MLP 모델 클래스를 정의합니다.- 입력층 (784 = 28 * 28)- 2개의 은닉층 (각각 ReLU 활성화 함수 포함)- 과적합 방지를 위한 드롭아웃(Dropout) 레이어 추가- 출력층 (10 = 클래스 개수)- `forward` 메소드에서 데이터 흐름 정의 (Flatten -> FC1 -> ReLU -> Dropout -> FC2 -> ReLU -> Dropout -> FC3)

In [ ]:
class MLP(nn.Module):    def __init__(self, input_size=784, hidden_size1=512, hidden_size2=256, num_classes=10, dropout_prob=0.3): # dropout 확률 조정 가능        super(MLP, self).__init__()        self.fc1 = nn.Linear(input_size, hidden_size1)        self.relu1 = nn.ReLU()        self.fc2 = nn.Linear(hidden_size1, hidden_size2)        self.relu2 = nn.ReLU()        self.fc3 = nn.Linear(hidden_size2, num_classes)        self.dropout = nn.Dropout(dropout_prob) # 드롭아웃 확률 설정    def forward(self, x):        # 입력을 평탄화(flatten)        x = x.view(-1, 28*28) # (Batch_size, 1, 28, 28) -> (Batch_size, 784)        # 첫 번째 은닉층        x = self.fc1(x)        x = self.relu1(x)        x = self.dropout(x) # 드롭아웃 적용        # 두 번째 은닉층        x = self.fc2(x)        x = self.relu2(x)        x = self.dropout(x) # 드롭아웃 적용        # 출력층 (CrossEntropyLoss 사용 시 Softmax 불필요)        x = self.fc3(x)        return x

## 7. 모델, 손실 함수, 옵티마이저 초기화

In [ ]:
# 모델 초기화 및 device로 이동model = MLP().to(device)print("\nModel Architecture:")print(model)# 손실 함수 정의 (다중 클래스 분류)criterion = nn.CrossEntropyLoss()# 옵티마이저 정의 (Adam 사용)optimizer = optim.Adam(model.parameters(), lr=learning_rate)# (Optional) Learning Rate Scheduler 추가 (예: StepLR)# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) # 5 에포크마다 학습률 0.1배 감소

## 8. 학습 및 평가 함수 정의

In [ ]:
# 학습 함수 정의def train_one_epoch(model, train_loader, criterion, optimizer, device, epoch, num_epochs):    model.train() # 모델을 학습 모드로 설정    running_loss = 0.0    correct = 0    total = 0    start_time = time.time()    for i, (images, labels) in enumerate(train_loader):        images, labels = images.to(device), labels.to(device)        # Forward pass        outputs = model(images)        loss = criterion(outputs, labels)        # Backward and optimize        optimizer.zero_grad()        loss.backward()        optimizer.step()        running_loss += loss.item()        # 정확도 계산        _, predicted = torch.max(outputs.data, 1)        total += labels.size(0)        correct += (predicted == labels).sum().item()        # 배치별 진행 상황 출력 (옵션)        # if (i+1) % 100 == 0:        #     print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')    end_time = time.time()    epoch_time = end_time - start_time    train_loss = running_loss / len(train_loader)    train_acc = 100 * correct / total    return train_loss, train_acc, epoch_time

In [ ]:
# 평가 함수 정의def evaluate(model, test_loader, criterion, device):    model.eval() # 모델을 평가 모드로 설정    running_loss = 0.0    correct = 0    total = 0    all_preds = []    all_labels = []    with torch.no_grad(): # 평가 시에는 그래디언트 계산 불필요        for images, labels in test_loader:            images, labels = images.to(device), labels.to(device)            outputs = model(images)            loss = criterion(outputs, labels)            running_loss += loss.item()            _, predicted = torch.max(outputs.data, 1)            total += labels.size(0)            correct += (predicted == labels).sum().item()            all_preds.extend(predicted.cpu().numpy())            all_labels.extend(labels.cpu().numpy())    test_loss = running_loss / len(test_loader)    test_acc = 100 * correct / total    return test_loss, test_acc, all_preds, all_labels

## 9. 모델 학습 및 평가 실행- 정의된 함수들을 사용하여 지정된 에포크 수만큼 모델을 학습시키고, 매 에포크마다 테스트 데이터셋으로 성능을 평가합니다.- 학습 및 테스트 손실, 정확도를 기록하여 시각화에 사용합니다.

In [ ]:
print("\nStarting training process...")train_losses = []train_accs = []test_losses = []test_accs = []total_start_time = time.time()for epoch in range(num_epochs):    train_loss, train_acc, epoch_time = train_one_epoch(model, train_loader, criterion, optimizer, device, epoch, num_epochs)    test_loss, test_acc, _, _ = evaluate(model, test_loader, criterion, device)    # (Optional) Update learning rate using scheduler    # scheduler.step()    # current_lr = scheduler.get_last_lr()[0]    train_losses.append(train_loss)    train_accs.append(train_acc)    test_losses.append(test_loss)    test_accs.append(test_acc)    print(f'Epoch [{epoch+1}/{num_epochs}], Time: {epoch_time:.2f}s, '          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '          f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')#, LR: {current_lr:.6f}') # If using schedulertotal_end_time = time.time()print(f"\nTraining finished. Total time: {total_end_time - total_start_time:.2f} seconds")

## 10. 모델 저장 (선택 사항)

In [ ]:
# 학습된 모델의 가중치를 파일로 저장model_save_path = 'fashion_mnist_mlp_final.pth'torch.save(model.state_dict(), model_save_path)print(f"\nModel state dictionary saved to {model_save_path}")# 모델 로드 예시 (필요 시 사용)# model_loaded = MLP().to(device)# model_loaded.load_state_dict(torch.load(model_save_path))# model_loaded.eval() # 평가 모드로 설정 필수

## 11. 학습 결과 시각화: 손실 및 정확도 그래프- 에포크별 학습 및 테스트 손실 변화를 그래프로 시각화합니다.- 에포크별 학습 및 테스트 정확도 변화를 그래프로 시각화합니다.

In [ ]:
print("\nVisualizing training results...")plt.figure(figsize=(12, 5))plt.subplot(1, 2, 1)plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss')plt.plot(range(1, num_epochs + 1), test_losses, label='Test Loss')plt.xlabel('Epoch')plt.ylabel('Loss')plt.title('Training & Test Loss')plt.legend()plt.grid(True)plt.subplot(1, 2, 2)plt.plot(range(1, num_epochs + 1), train_accs, label='Train Accuracy')plt.plot(range(1, num_epochs + 1), test_accs, label='Test Accuracy')plt.xlabel('Epoch')plt.ylabel('Accuracy (%)')plt.title('Training & Test Accuracy')plt.legend()plt.grid(True)plt.tight_layout()plt.show()

## 12. 최종 모델 평가 및 혼동 행렬 시각화- 학습이 완료된 모델을 사용하여 테스트 데이터셋 전체에 대한 최종 예측을 수행합니다.- 실제 레이블과 예측 레이블을 사용하여 혼동 행렬(Confusion Matrix)을 계산하고 `seaborn`을 이용해 히트맵으로 시각화합니다.

In [ ]:
print("\nCalculating final confusion matrix...")# 최종 모델 평가 (이미 학습 루프 마지막에 계산되었지만, 명시적으로 다시 실행 가능)final_test_loss, final_test_acc, all_preds, all_labels = evaluate(model, test_loader, criterion, device)print(f"Final Test Accuracy: {final_test_acc:.2f}%")# 혼동 행렬 계산conf_matrix = confusion_matrix(all_labels, all_preds)# 혼동 행렬 시각화plt.figure(figsize=(10, 8))sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',            xticklabels=classes, yticklabels=classes, annot_kws={"size": 10})plt.xlabel('Predicted Label')plt.ylabel('True Label')plt.title('Confusion Matrix for Fashion MNIST')plt.show()

## 13. 예측 예시 시각화- 테스트 데이터셋에서 일부 이미지를 가져와 모델의 예측 결과를 실제 레이블과 함께 시각화합니다.- 예측이 맞으면 녹색, 틀리면 빨간색으로 제목 색상을 표시합니다.

In [ ]:
# 예측 예시 시각화 함수def show_prediction_examples(model, test_loader, classes, device, num_examples=15):    model.eval()    data_iter = iter(test_loader)    images, labels = next(data_iter) # Get one batch    images_subset = images[:num_examples].to(device)    labels_subset = labels[:num_examples]    with torch.no_grad():        outputs = model(images_subset)        _, predicted = torch.max(outputs, 1)    num_rows = (num_examples + 4) // 5 # 5열 기준    fig, axes = plt.subplots(num_rows, 5, figsize=(12, 3 * num_rows))    axes = axes.flatten()    for i in range(num_examples):        ax = axes[i]        # 정규화 복원 후 이미지 표시        img_np = images_subset[i].cpu().numpy()        img_np = img_np * 0.3530 + 0.2860 # Denormalize        img_np = np.clip(img_np, 0, 1) # 값 범위를 0~1로 제한        ax.imshow(np.transpose(img_np, (1, 2, 0)).squeeze(), cmap='gray')        pred_class = classes[predicted[i].item()]        true_class = classes[labels_subset[i].item()]        color = 'green' if predicted[i] == labels_subset[i] else 'red'        ax.set_title(f'True: {true_class}\nPred: {pred_class}', color=color, fontsize=10)        ax.axis('off')    # 남는 subplot 비활성화    for j in range(i + 1, len(axes)):        axes[j].axis('off')    plt.tight_layout()    plt.suptitle('Prediction Examples (Green: Correct, Red: Incorrect)', y=1.02)    plt.show()

In [ ]:
# 예측 예시 시각화 실행print("\nShowing prediction examples...")show_prediction_examples(model, test_loader, classes, device, num_examples=15)

## 코드 설명이 코드는 다음과 같은 주요 부분으로 구성되어 있습니다:- **데이터 준비**: Fashion MNIST 데이터셋 로드, (선택적) 데이터 증강 적용, 정규화 및 DataLoader를 이용한 배치 처리 설정- **MLP 모델 구현**: 입력층 (784 유닛, 28x28 이미지를 펼침), 첫 번째 은닉층 (512 유닛 + ReLU 활성화), 두 번째 은닉층 (256 유닛 + ReLU 활성화), 출력층 (10 유닛, 각 클래스에 대한 점수), 드롭아웃 적용으로 과적합 방지- **학습 과정**: CrossEntropyLoss 손실 함수 사용, Adam 옵티마이저로 학습, 지정된 에폭 동안 학습 진행 및 성능 기록- **결과 시각화**:- 학습/테스트 데이터 샘플 이미지- 학습/테스트 손실 및 정확도 그래프- 혼동 행렬(Confusion Matrix)을 통한 클래스별 성능 분석- 실제 이미지와 모델의 예측 결과 비교 시각화이 MLP 모델은 Fashion MNIST 데이터셋에서 일반적으로 약 85-90% 정도의 테스트 정확도를 달성할 수 있습니다. 모델 구조나 하이퍼파라미터를 조정하여 성능을 더 향상시킬 수도 있습니다.